<a href="https://colab.research.google.com/github/dainis-boumber/av/blob/master/classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ULMFiT + Siamese Network for Sentence Vectors
## Part Three: Classifying

The second notebook created a new language model from the SNLI dataset.
This notebook will adapt that model to predicting the SNLI category for sentence pairs.
The model will be used as a sentence encoder for a Siamese Network that builds sentence vectors that are feed into a classifier network.

In [0]:
import fastai
from fastai.text import *
import html

import json
import html
import re
import pickle
from collections import Counter
import random
import pandas as pd
import numpy as np
from pathlib import Path
import sklearn
from sklearn import model_selection
from functools import partial
from collections import Counter, defaultdict
import random

import numpy as np
import torch
import torch.nn as nn
import torch.utils 
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
from torch.utils.data import dataset, dataloader
import torch.optim as optim
import torch.nn.functional as F

import time
import math
import sys
import data
import joblib

token_files = './data/PAN14/tokens/'
model_files = './data/PAN14/models/'
TRAINDATAPATH = "./data/PAN14/pan14_train_english-essays/"
TESTDATAPATH = "./data/PAN14/pan14_test01_english-essays/"
FNAMES = ['known01','known02','known03','known04','known05', 'unknown']
KCOLS=['known01','known02','known03','known04','known05']
LABELCOL="answer"
UNKOWN="unknown"

In [0]:
doc_pairs_train=joblib.load(f'{model_files}traindf.pkl')
doc_pairs_val=joblib.load(f'{model_files}valdf.pkl')
doc_pairs_test=joblib.load(f'{model_files}testdf.pkl')

In [0]:
data_lm = TextLMDataBunch.load(model_files)

In [0]:
data_clf = TextClasDataBunch.from_df(model_files, doc_pairs_train, doc_pairs_val, doc_pairs_test,
                                vocab=data_lm.train_ds.vocab, bs=32,
                                  text_cols=['known', 'unknown'], label_cols=['label'], mark_fields=True)



In [0]:
data_clf.save()          
data_clf = TextClasDataBunch.load(model_files, bs=32)

In [0]:
learn = text_classifier_learner(data_clf, drop_mult=0.5)
learn.load_encoder('ft_enc')

In [0]:
data_clf.show_batch()

text,target
xxbos xxfld 1 ﻿ xxmaj little xxmaj women xxmaj by xxmaj louisa xxup m. \n xxmaj alcott xxmaj an ideal female world without any xxunk xxunk is the setting of xxmaj louisa xxup m. \n xxmaj alcott 's novel xxmaj little women.x_bodthe novel presents a romantic picture of a xxunk home where the family members love and care for each other more than anything in the xxunk the father absent,1
"xxbos xxfld 1 ﻿ xxmaj little xxmaj women by xxup l. \n xxmaj alcott xxmaj the critic xxmaj xxunk xxmaj xxunk says about xxmaj edna xxmaj pontellier in xxmaj the xxmaj awakening by xxmaj xxunk xxmaj chopin that she "" appears to reject the domestic empire of the mother and the sororal world of women 's culture "" ( p xxunk will in this essay discuss these two phenomena -",1
"xxbos xxfld 1 xxunk xxmaj introduction and method xxmaj have you ever wondered about how words come to xxunk are they actually xxunk interest grew when i started to read about the different word xxunk were a lot of them and i wanted to know more about it.x_bodmy aim with this essay is first to define the ways that new words can be formed , and than use this",0
"xxbos xxfld 1 ﻿ xxmaj shakespeare 's xxmaj julius xxmaj caesar xxmaj after "" a very xxunk night to honest men "" ( xxup xxunk ) - a night of xxunk - xxmaj cassius comes to xxmaj brutus 's house in the company of xxmaj casca , xxmaj cinna , xxmaj decius , xxmaj trebonius and xxunk is early in the morning the day for the assassination and they have",1
"xxbos xxfld 1 ﻿ xxmaj is xxmaj english a xxmaj germanic or a xxmaj romance language?x_bod1.x_bodthere are people like xxmaj jean - xxmaj marc xxmaj gachelin , the xxmaj professor of xxmaj english at the xxmaj university of xxmaj rouen , asking whether xxmaj english is a xxmaj romance language or not , while "" xxmaj english behaves like a xxmaj romance language "" , according to the article """,0


In [0]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy
1,0.618289,0.654066,0.613900


In [0]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(5e-3/2., 5e-3))

epoch,train_loss,valid_loss,accuracy
1,0.596755,0.729767,0.525097


In [0]:
learn.unfreeze()
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))


epoch,train_loss,valid_loss,accuracy


RuntimeError: CUDA out of memory. Tried to allocate 40.50 MiB (GPU 0; 7.93 GiB total capacity; 3.47 GiB already allocated; 34.94 MiB free; 52.29 MiB cached)